#Program to compute expectation values and check wheter the the state created at the lab approaches to the singlet

The space is spanded into $|N_{-1},N_0,N_{+1}\rangle$ three mode symmetric picture, on wich the particle number is conserved, $N_{-1}+N_0+N_{+1}=N$.

In total one has to deal with $N(N+1)/2$ states basis. And also all the possible operators must be spanded by pairs of $a_\alpha^\dagger a_\beta$ operators, where $\alpha$ and $\beta$ can take $\{-1,0,+1\}$ values in order to maintain the particle number constant. Thos operators must be deffined in order to satisfy the commutation relations for bosonic systems, $[a_\alpha,a_\beta^\dagger]=\delta_{\alpha,\beta}$.

#### Initialize the environtment to save data and such a things

In [ ]:
using HDF5

## Create the basis and all the $a_\alpha^\dagger a_\beta$ operators

The basis is from upside down $|0,0,N\rangle,|0,1,N-1\rangle,|0,2,N-2\rangle,\dots,|1,0,N-1\rangle,\dots,|N,0,0\rangle$.

There are 3 unique conminations of $a_\alpha^\dagger a_\alpha$ the rest can be computed with the hermitian conjugate.

In [1]:
function createAnnihilate(α, β, N)
    
    dim = Int64((N+1)*(N+2)/2)
        
    if α == β
        
        diagonal = Float64[]
        
        if α == -1
                        
            for i in range(0,N+1)
                
                diagonal = [diagonal; fill(i,N+1-i)]
                
            end
                        
        elseif α == 0
            
            for i in range(0,N+1)
            
                for j in range(0,N+1-i)
                    
                    diagonal = [diagonal; j]
                    
                end
            
            end

        else
            
            for i in range(0,N+1)
            
                for j in range(0,N+1-i)
                    
                    diagonal = [diagonal; N-i-j]
                    
                end
            
            end

        end
        
        operator = diagm(diagonal)
        
    else
        
        operator = fill(0.,dim,dim)
        
        index = 0
        
        if α == -1
            
            if β == 0
                
                for i in range(0,N+1)

                    for j in range(0,N+1-i)

                        index += 1

                        if i != 0

                            operator[index, index-N-1+i] = sqrt(j+1)*sqrt(i)

                        end

                    end 

                end
                                
            else
                
                for i in range(0,N+1)
                    
                    for j in range(0,N+1-i)
                        
                        index += 1

                        if i != 0

                            operator[index, index-N-2+i] = sqrt(N-i+1-j)*sqrt(i)

                        end
                        
                    end 
                    
                end
                
            end
            
        elseif α == 0 
            
            if β == -1
                
                operator = createAnnihilate(-1,0,N)'
                
            else
                
                for i in range(0,N+1)
                    
                    for j in range(0,N+1-i)
                        
                        index += 1
                        
                        if j != 0
                        
                            operator[index, index-1] = sqrt(N-i-j+1)*sqrt(j)
                        
                        end
                        
                    end 
                    
                end
                
            end
            
        else
            
            if β == -1
                
                operator = createAnnihilate(-1,1,N)'
                
            else
                
                operator = createAnnihilate(0,1,N)'
                
            end
            
        end
        
    end
    
    operator
    
end

createAnnihilate (generic function with 1 method)

There are only 3 $a_\alpha a_\alpha^\dagger$, since the subindex must be equal. If not they become the same as the function above because when the indexes are not equal the annihilation and creation operators commute.

In [2]:
annihilateCreate(α, N) = eye(Int((N+1)*(N+2)/2)) + createAnnihilate(α,α,N)

annihilateCreate (generic function with 1 method)

In [3]:
@time begin
    N = 4
    for i in range(-1,3)
        for j in range(-1,3)
            1im * createAnnihilate(i,j,N)
        end
    end
end

## Test the operators

For $N$ particlest in the state $|N,0,0>$ and $|0,0,N>$ the $\langle J_z \rangle = \pm N$, $\langle J_x \rangle = \langle J_y \rangle = 0$ and ...

In [4]:
Jz(N) = createAnnihilate(1,1,N) - createAnnihilate(-1,-1,N)
Jx(N) = 1/sqrt(2) * (createAnnihilate(-1,0,N) + createAnnihilate(0,-1,N) +
    createAnnihilate(1,0,N) + createAnnihilate(0,1,N))
Jy(N) = 1im/sqrt(2) * (-createAnnihilate(-1,0,N) + createAnnihilate(0,-1,N) +
    createAnnihilate(1,0,N) - createAnnihilate(0,1,N))

Jy (generic function with 1 method)

0.272064 seconds (208.16 k allocations: 9.872 MB, 1.47% gc time)


In [5]:
@time begin
    N = 4
    Jx(N)
    Jy(N)
    Jz(N)
end;

  0.063625 seconds (46.84 k allocations: 2.408 MB)


Autmatize the creation of singular states such as the singlet, the totally polarized states, the planar Dicke state for spin 1 etc...

### Totally polarized states or differently populated states.

$N$ the particle number must be conserved. Let deffine a function where to create basis vectors out of the populations of the 3 modes.

In [6]:
function ket(k, l, m)
    
    N = k+l+m
    
    state = fill(0,Int((N+1)*(N+2)/2))
    
    if k == 0
        
        state[l+1] = 1
        
    else
        
        state[Int(l+1+k*(N+1-(k-1)/2))] = 1
        
    end
    
    state
    
end

ket (generic function with 1 method)

## The problem

Be an initial state shuch as $|0,N,0\rangle$ and evolbe under a hamiltonian of the following type,
$$H=J_x^2+J_y^2$$
Which is the value of the expectation value for $\sum (\Delta J_l)^2 - N$?

In [7]:
function phasem(phi,N)
        
    diagonal = Complex{Float64}[]

    for i in range(0,N+1)

        for j in range(0,N+1-i)

            diagonal = [diagonal; exp(1im*phi*j)]

        end

    end

    diagm(diagonal)

end

function singletWitness(phi,t,N)
    
    init_state = ket(0,N,0)
    
    H = Jx(N)^2+Jy(N)^2+Jz(N)^2
                
    state = phasem(phi,N)*expm(-1im*t*H)*init_state
    
    real(state'*(Jx(N)^2 + Jy(N)^2 +Jz(N)^2)*state)-    
        abs2(real(state'*Jx(N)*state))-abs2(real(state'*Jy(N)*state))-abs2(real(state'*Jz(N)*state))-N
    
end

singletWitness (generic function with 1 method)

Create a table for efficienci of the code.

|`singletWitness`|(0,0,4)   |(0.5,0,4) |(0.5,2.3,4)|(0.5,2.3,10)|(0.5,2.3,25)|(0.5,2.3,60)|N>60|
|----------------|---------:|---------:|----------:|-----------:|-----------:|-----------:|---:|
| time           |0.67 ms   |0.7 ms    |1.1 ms     |6 ms        |0.18 s      |17 s        |fail|
| allocation     |1.9 k     |1.9 k     |1.95 k     |7.55 k      |37.8 k      |229.2 k     |fail|
|                |351.109 KB|351.109 KB|451.55 KB  |6.98 MB     |193.482 MB  |5.541 GB    |fail|
| gc time        |          |          |           |            |4.04%       |4.2%        |fail|

In [18]:
@time singletWitness(0.5, 2.3, 10);

  0.004936 seconds (7.55 k allocations: 6.976 MB)


Consider using sparse matrices to save memory allocations.

###Run the function and save the data

In [ ]:
function runAndSave()
    N = [2,4,8,16,20]
    phases = linspace(0,pi,30)
    fileName = "resultERP.h5"
    for 
end